In [20]:
import numpy as np
import pandas as pd
import plotly_express as px

In [3]:
measurements = pd.read_csv('../../nrao_measurements.csv')
measurements = measurements.set_index('project_code')
measurements.head()

,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,,,,,
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1,0.24,90.500,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1,0.23,90.815,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1,0.23,91.805,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1,0.23,93.005,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1,0.94,218.060,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...


There are 44,230 mesurements made across 3,628 'line' projects.

In [6]:
line_measurements = measurements.loc[measurements.fs_type == 'line']
line_measurements.fs_type.value_counts()

fs_type
line    44230
Name: count, dtype: int64

In [14]:
project_measurements = line_measurements.groupby(line_measurements.index)\
    .project_title.count()\
        .sort_values(ascending=False)\
        .to_frame()
project_measurements.columns = ['measurement_count']
project_measurements.head()

,measurement_count
project_code,
2017.1.00161.L,289
2017.1.00886.L,283
2021.2.00052.S,265
2023.1.00963.S,253
2022.1.00224.S,188


The number of measurements by project is heavily skewed right.

In [19]:
project_measurements.describe()

,measurement_count
count,3628.000000
mean,12.191290
std,19.324441
min,1.000000
25%,4.000000
50%,7.000000
75%,13.000000
max,289.000000


In [24]:
px.box(project_measurements, orientation='h', title='Number of measurments by project')

Create data frame of outlier projects.

In [43]:
iqr = project_measurements.describe().loc['75%'].values - project_measurements.describe().loc['25%'].values
upper_fence = (project_measurements.describe().loc['75%'].values + (1.5*iqr))[0]
upper_fence

26.5

In [47]:
outliers = project_measurements[project_measurements.measurement_count > upper_fence]
outliers.describe()

,measurement_count
count,344.000000
mean,55.752907
std,39.101572
min,27.000000
25%,32.000000
50%,42.000000
75%,65.000000
max,289.000000


In [49]:
outliers.head()

,measurement_count
project_code,
2017.1.00161.L,289
2017.1.00886.L,283
2021.2.00052.S,265
2023.1.00963.S,253
2022.1.00224.S,188


Number of outlier projects

In [70]:
outliers.shape[0]

344

Proportion of outliers to line projects

In [57]:
round(outliers.shape[0]/project_measurements.shape[0], 4)

0.0948

Number of remaining projects in data after removing outliers

In [58]:
project_measurements.shape[0] - outliers.shape[0]

3284

Number of measurements associated with outliers

In [66]:
outliers.sum().values[0]

19179

Number of measurments after removing outliers

In [68]:
project_measurements.sum().values[0] - outliers.sum(0).values[0]

25051

Ratio of outlier measurements to total measurements

In [69]:
outliers.sum(0).values[0]/project_measurements.sum().values[0]

0.4336197151254804

9.48% of projects are contributing 43.36% of measurements if we do not remove outliers